In [0]:
# Download PyDrive
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download training files
import os
local_download_path = os.path.expanduser('/content')
try:
  os.makedirs(local_download_path)
except: pass
file_list = drive.ListFile({'q': "'1DZrRBUX2BBXoT6BIQoQB1cWFtq4LYKVL' in parents"}).GetList()
for f in file_list:
  fname = os.path.join(local_download_path, f['title'])
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  
!unzip yolo.zip

In [0]:
# Clone darknet
!git clone https://github.com/AlexeyAB/darknet/
  
# Move files from yolo to darknet
%cd darknet
!bash -c 'mv /content/yolo/data/* /content/darknet/data/'
!bash -c 'mv /content/yolo/cfg/* /content/darknet/cfg/'
!bash -c 'mv /content/yolo/yolov3-strawberry.weights /content/darknet/'
!bash -c 'mv /content/yolo/yolov3-tiny-strawberry.weights /content/darknet/'

# Enable OpenCV and GPU in Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile

# make darknet
!make

In [0]:
# Test detection on custom image
!./darknet detector test data/strawberry.data cfg/yolov3-tiny-strawberry.cfg yolov3-tiny-strawberry.weights data/strawberry/0508.jpg -dont_show -ext_output < data/test.txt > result.txt